<a href="https://colab.research.google.com/github/Danr17/TensorFlow_coding/blob/master/Computer_Vision/Flowers_TF_Linear%2C_DNN_%26_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credits to Google training,  Advanced Machine Learning with Tensorflow on GCP

In [19]:
import numpy as np
import shutil
import tensorflow as tf
print(tf.__version__)
#tf.logging.set_verbosity(tf.logging.INFO)

1.12.0


In [0]:
LIST_OF_LABELS='daisy,dandelion,roses,sunflowers,tulips'.split(',')
HEIGHT=299
WIDTH=299
NUM_CHANNELS=3
NCLASSES=5

**Define the Linear model low level or using tf.layers API**

In [0]:
def linear_model(img, mode, hparams):
  X=tf.reshape(img,[-1, HEIGHT*WIDTH*NUM_CHANNELS])
  ylogits = tf.layers.dense(X,NCLASSES,activation = None)
  return ylogits, NCLASSES

Result for Linear: ?

**Define the DNN model**

In [0]:
def dnn_model(img, mode, hparams):
  X= tf.reshape(img,[-1,HEIGHT*WIDTH*NUM_CHANNELS])
  h1 = tf.layers.dense(X, 300, activation = tf.nn.relu)
  h2 = tf.layers.dense(h1, 100, activation = tf.nn.relu)
  h3 = tf.layers.dense(h2, 30, activation = tf.nn.relu)
  ylogits = tf.layers.dense(h3, NCLASSES, activation = None)
  return ylogits, NCLASSES

Results for DNN: ?

**Define DNN Dropout model**

In [0]:
def dnn_dropout_model(img, mode, hparams):
  dprob = hparams.get('dprob', 0.1)
  
  X= tf.reshape(img,[-1,HEIGHT*WIDTH*NUM_CHANNELS])
  h1 = tf.layers.dense(X, 300, activation = tf.nn.relu)
  h2 = tf.layers.dense(h1, 100, activation = tf.nn.relu)
  h3 = tf.layers.dense(h2, 30, activation = tf.nn.relu)
  h3d = tf.layers.dropout(h3, rate=dprob, training=(mode == tf.estimator.ModeKeys.TRAIN))
  ylogits = tf.layers.dense(h3d, NCLASSES, activation = None)
  return ylogits, NCLASSES

Results for DNN with DropOut: 

**Define CNN model**

In [0]:
def cnn_model(img, mode, haparams):
  ksize1 = hparams.get('ksize1', 5)
  ksize2 = hparams.get('ksize2', 5)
  nfil1 = hparams.get('nfil1', 10)
  nfil2 = hparams.get('nfil2', 20)
  dprob = hparams.get('drop', 0.25)
  
  c1 = tf.layers.conv2d(img, filters=nfil1, kernel_size=ksize1, strides=1, padding='same', activation=tf.nn.relu) 
  p1 = tf.layers.max_pooling2d(c1, pool_size=2, strides=2) 
  c2 = tf.layers.conv2d(p1, filters=nfil2, kernel_size=ksize2, strides=1, padding='same', activation=tf.nn.relu) 
  p2 = tf.layers.max_pooling2d(c2, pool_size=2, strides=2) 
  
  outlen = p2.shape[1]*p2.shape[2]*p2.shape[3] 
  p2flat = tf.reshape(p2, [-1, outlen])
  
  h3 = tf.layers.dense(p2flat, 300, activation=tf.nn.relu)
  h3d = tf.layers.dropout(h3, rate=dprob, training=(mode == tf.estimator.ModeKeys.TRAIN))
  ylogits = tf.layers.dense(h3d, NCLASSES, activation=None)
  return ylogits, NCLASSES

Results for CNN: ?

**Write Input Functions**

In [0]:
def read_and_preprocess_with_augment(image_bytes, label=None):
  return read_and_preprocess(image_bytes, label, augment=True)

In [0]:
def read_and_preprocess(image_bytes, label=None, augment= False):
  # decode the image, end up with pixel values witch are in -1, 1 range
  image = tf.image.decode_jpeg(image_bytes, channels=NUM_CHANNELS)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32) # 0 - 1
  image = tf.expand_dims(image, 0) # resize with batches
  
  if augment:
    image = tf.image.resize_bilinear(image, [HEIGHT+10, WIDTH+10], align_corners=False)
    image = tf.squeeze(image) # remove batch dimension
    image = tf.random_crop(image, [HEIGHT, WIDTH, NUM_CHANNELS])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=63.0/255.0)
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
  else:
    image = tf.image.resize_bilinear(image, [HEIGHT, WIDTH], align_corners=False)
    image = tf.squeeze(image) # remove batch dimension
    
  # pizel values are in [0,1] range, convert to [-1,1]
  image = tf.subtract(image, 0.5)
  image = tf.multiply(image, 2.0)
  return {'image': image}, label

In [0]:
def serving_input_fn():
  # handles one image at a time
  feature_placeholders = {'image_bytes': tf.placeholder(tf.string, shape())}
  image, _ = read_and_preprocess(tf.squeeze(feature_placeholders['image_bytes']))
  image['image'] = tf.expand_dims(image['image'], 0)
  return tf.estimator.export.ServingInputReceive(image, feature_placeholders)
  

In [0]:
def make_input_fn(csv_of_filenames, batch_size, mode, augment=False):
  def _input_fn():
    def decode_csv(csv_row):
      filename, label = tf.decode_csv(csv_row, record_defaults=[[''],['']])
      image_bytes = tf.read_file(filename)
      return image_bytes, label
    
    #create tf.data.dataset from filename
    dataset = tf.data.TextLineDataset(csv_of_filenames).map(decode_csv)
    
    if augment:
      dataset = dataset.map(read_and_preprocess_with_augment)
    else:
      dataset = dataset.map(read_and_preprocess)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None
      dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
      num_epochs = 1
    
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn

**Write a custom estimator to accomodate Linear, DNN and CNN models**

In [0]:
def image_classifier(features, labels, mode, params):
  model_functions = {'linear': linear_model, 'dnn': dnn_model, 'dnn_dropout': dnn_dropout_model, 'cnn': cnn_model}
  model_function = model_functions[params['model']]
  ylogits, nclasses = model_function(features['image'], mode, params)
  
  probabilities = tf.nn.softmax(ylogits)
  class_int = tf.cast(tf.argmax(probabilities, 1), tf.uint8)
  class_str = tf.gather(LIST_OF_LABELS, tf.cast(class_int, tf.int32))
  
  
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    # convert string label to int
    labels_table = tf.contrib.lookup.index_table_from_tensor(tf.constant(LIST_OF_LABELS))
    labels = labels_table.lookup(labels)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = ylogits, labels=tf.one_hot(labels, nclasses)))
    evalmetrics ={'accuracy': tf.metrics.accuracy(class_int, labels)}
                                                  
    if mode == tf.estimator.ModeKeys.TRAIN:
      # this is needed for batch normalization, but has no effct otherwise
      update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
      with tf.control_dependencies(update_ops):
        train_op = tf.contrib.layers.optimize_loss(loss, tf.train.get_global_step(), learning_rate=params['learning_rate'], optimizer="Adam")
    else:
      train_op = None
  else:
    loss = None
    train_op = None
    evalmetrics = None
  return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={"probabbilities": probabilities, "classid": class_int, "class": class_str},
        loss=loss,
        train_op=train_op,
        eval_metric_ops=evalmetrics,
        export_outputs={"classes": tf.estimator.export.PredictOutput({"probabilities": probabilities, "classid": class_int, "class": class_str})}
    )

In [0]:
def train_and_evaluate(output_dir, hparams):
  EVAL_INTERVAL = 300
  estimator = tf.estimator.Estimator(model_fn = image_classifier, params = hparams, config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL), model_dir = output_dir)
  train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(hparams['train_data_path'], hparams['batch_size'], mode = tf.estimator.ModeKeys.TRAIN, augment = hparams['augment']), max_steps = hparams['train_steps'])
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(hparams['eval_data_path'], hparams['batch_size'], mode = tf.estimator.ModeKeys.TRAIN), steps = None, exporters = exporter, start_delay_secs = EVAL_INTERVAL, throttle_secs = EVAL_INTERVAL)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [0]:
!gsutil -m cp -r gs://cloud-ml-data/img/flower_photos/* .

In [8]:
!ls

all_data.csv		 flowers_200_presplit.zip      roses.zip
daisy			 flowers_200_unlabeled.zip     sample_data
daisy.zip		 flowers_full_with_csv.zip     sunflowers
dandelion		 inception_v3_2016_08_28.ckpt  sunflowers.zip
dandelion.zip		 LICENSE.txt		       train_set.csv
dict.txt		 mnist			       tulips
eval_set.csv		 open_image_inception_v3.ckpt  tulips.zip
flowers_200_csv.zip	 README.txt
flowers_200_folders.zip  roses


In [0]:

#!sed -i 's/gs\:\/\/cloud\-ml\-data\/img\/flower_photos\//\/content\//' train_set.csv
#!sed -i 's/gs\:\/\/cloud\-ml\-data\/img\/flower_photos\//\/content\//' eval_set.csv

In [0]:
OUTDIR = 'flowers/learned'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh

hparams = {'model': 'linear','train_steps': 100, 'learning_rate': 0.01, 'batch_size': 100, 'train_data_path': 'train_set.csv', 'eval_data_path': 'eval_set.csv', 'augment': False}
train_and_evaluate(OUTDIR, hparams)